<a href="https://colab.research.google.com/github/ARU-Bioinformatics/ARU-Bioinf-CMA-2021/blob/main/week_6/relational_database_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import sqlite3
import time

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('bioinformatics.db')
c = conn.cursor()

# Create tables in the database
c.execute('''CREATE TABLE IF NOT EXISTS Patients (
                PatientID INTEGER PRIMARY KEY NOT NULL,
                Name TEXT NOT NULL,
                Age INTEGER NOT NULL);''')

c.execute('''CREATE TABLE IF NOT EXISTS Tests (
                TestID INTEGER PRIMARY KEY NOT NULL,
                TestName TEXT NOT NULL);''')

c.execute('''CREATE TABLE IF NOT EXISTS Results (
                ResultID INTEGER PRIMARY KEY NOT NULL,
                PatientID INTEGER,
                TestID INTEGER,
                Result TEXT,
                FOREIGN KEY(PatientID) REFERENCES Patients(PatientID),
                FOREIGN KEY(TestID) REFERENCES Tests(TestID));''')
conn.commit()

# Insert sample data into the tables
# Note: In a real scenario, ensure that IDs are unique and increment accordingly
patients_data = [(1, 'John Doe', 30), (2, 'Jane Smith', 25), (3, 'Emily Evans', 40)]
c.executemany('INSERT INTO Patients (PatientID, Name, Age) VALUES (?, ?, ?);', patients_data)

tests_data = [(1, 'Blood Test'), (2, 'DNA Analysis')]
c.executemany('INSERT INTO Tests (TestID, TestName) VALUES (?, ?);', tests_data)

results_data = [(1, 1, 1, 'Normal'), (2, 2, 2, 'Variant Found'), (3, 3, 1, 'Slightly Elevated')]
c.executemany('INSERT INTO Results (ResultID, PatientID, TestID, Result) VALUES (?, ?, ?, ?);', results_data)
conn.commit()

# Query data without an index
start_time = time.time()
for _ in range(1000):  # Simulate larger dataset by repeating the query
    c.execute('SELECT Name FROM Patients WHERE Age > 35;')
    results = c.fetchall()
end_time = time.time()
print(f"Query time without index: {end_time - start_time:.5f} seconds")

# Add an index on the Age column in the Patients table
c.execute('CREATE INDEX IF NOT EXISTS age_index ON Patients(Age);')
conn.commit()

# Query data with an index
start_time = time.time()
for _ in range(1000):  # Simulate larger dataset by repeating the query
    c.execute('SELECT Name FROM Patients WHERE Age > 35;')
    results = c.fetchall()
end_time = time.time()
print(f"Query time with index: {end_time - start_time:.5f} seconds")

# Close the database connection
conn.close()


Query time without index: 0.01104 seconds
Query time with index: 0.01459 seconds


# Explanation:
*  The script starts by connecting to an SQLite database called bioinformatics.db.

*  If the database doesn't exist, SQLite creates it.

*  It then creates three tables: Patients, Tests, and Results, using the CREATE TABLE IF NOT EXISTS command to avoid errors if the tables already exist.

*  Sample data is inserted into each table. This step uses the executemany() method to insert multiple rows at once.

*  The script executes a query to select patient names where the age is greater than 35, measuring the time it takes to complete this operation without an index. This is repeated 1000 times to simulate a larger dataset and make the time difference more noticeable.

*  An index is created on the Age column of the Patients table to improve the performance of queries that filter by age.

*  The same query is executed again, this time measuring how the index affects performance. Finally, the script closes the connection to the database.